## Load libraries

In [1]:
using Distributions
using IterTools
using JuMP
using GLPK
using Random

## Define key parameters

In [2]:
num_items = 100
num_dimensions = 5

items = ["item_$x" for x in 1:num_items]
dimensions = ["dim_$x" for x in 1:num_dimensions];

## Generate (random) input data

In [3]:
normal_dist = Normal(10.0, 1.0)

item_value = Dict(zip(items, round.(rand(normal_dist, num_items), digits = 2)))
item_space = Dict(zip(product(items, dimensions), round.(rand(normal_dist, num_dimensions, num_items), digits = 2)));

In [4]:
knapsack_dist = Normal(5*num_items, 0.5*num_items)

knapsack_space = Dict(zip(dimensions, round.(rand(knapsack_dist, num_dimensions), digits = 2)));

## Define model via JuMP

In [5]:
@time begin
    model = Model()
    set_optimizer(model, GLPK.Optimizer)
    @variable(model, x[items], Bin)
    @constraint(model, dimension_constraint[dimension in dimensions], sum(item_space[item, dimension] * x[item] for item in items) <= knapsack_space[dimension])
    @objective(model, Max, sum(item_value[item] * x[item] for item in items));
end;

  3.607006 seconds (2.67 M allocations: 140.563 MiB, 1.09% gc time, 99.44% compilation time)


## Solve the model

In [6]:
@time optimize!(model)

  2.877703 seconds (3.39 M allocations: 185.832 MiB, 1.50% gc time, 99.57% compilation time)


## Inspect the results

In [7]:
num_items = sum(1 for item in items if value.(x[item]) > 0.5)
total_value = round(sum(item_value[item] for item in items if value.(x[item]) > 0.5), digits = 2)
total_space = Dict()
space_left = Dict()
for dimension in dimensions
    total_space[dimension] = round(sum(item_space[item, dimension] for item in items if value.(x[item]) > 0.5), digits = 2)
    space_left[dimension] = round(knapsack_space[dimension] - total_space[dimension], digits = 2)
end

print("Carry $num_items items of total value $total_value")
println()
println("Space used:")
println(total_space)
println("Space left:")
println(space_left)

Carry 46 items of total value 491.32
Space used:
Dict{Any, Any}("dim_3" => 465.48, "dim_1" => 453.23, "dim_5" => 451.72, "dim_4" => 448.51, "dim_2" => 436.14)
Space left:
Dict{Any, Any}("dim_3" => 43.42, "dim_1" => 11.04, "dim_5" => 49.4, "dim_4" => 65.21, "dim_2" => 0.0)
